In [4]:
from _labels import *
from src._cnn_config import *
from pathlib import Path
import pandas as pd

In [2]:
brainparts = []
pathogens = []
val_foldlist = []
aurocs = []
f1s = []
accs = []
prcs = []

for part_to_segment in labels:
    part_to_segment = part_to_segment.replace('-', '_')
    partlogdir = homedir + '/TrainLogs/' + part_to_segment.upper() + 'LOGS'

    for pathogen in pathogenlist:
        pathogenlogdir = partlogdir + '/' + str(pathogen)

        for val_fold in range(num_val_folds):
                valfolddir = pathogenlogdir + '/val_fold_' + str(val_fold)

                val_best_log_path = Path(valfolddir + '/test_log.csv')
                df_val = pd.read_csv(val_best_log_path)
                try:
                    auroc = float(df_val.loc[0, 'roc_auc'])
                except:
                    print('No AUROC for ' + part_to_segment)
                    break

                acc = float(df_val.loc[0, 'acc'])

                val_best_confusion_log_path = Path(valfolddir + '/test_confusion_log.csv')
                df_valconf = pd.read_csv(val_best_confusion_log_path)

                TP = int(df_valconf.loc[0, 'TP'])
                FN = int(df_valconf.loc[0, 'FN'])
                FP = int(df_valconf.loc[0, 'FP'])
                
                prc = TP/(TP+FP) if (TP + FP) != 0 else 0
                f1 = TP/(TP + (FP + FN)/2) if (TP + (FP + FN)/2) != 0 else 0
                
                brainparts.append(part_to_segment)
                pathogens.append(pathogen)
                val_foldlist.append(val_fold)
                aurocs.append(auroc)
                f1s.append(f1)
                accs.append(acc)
                prcs.append(prc)

df_test_metrics = pd.DataFrame({'Brainpart': brainparts, 'Pathogen': pathogens, 'Test_Fold': val_foldlist, 'AUROC': aurocs, 'F1_Score': f1s, 'Acc': accs, 'Prc': prcs})

# Display Dataframe
df_test_metrics.head()

,Brainpart,Pathogen,Test_Fold,AUROC,F1_Score,Acc,Prc
0,Cerebellum_White_Matter,0,0,0.401786,0.222222,0.125000,0.125
1,Cerebellum_White_Matter,0,1,0.142857,0.000000,0.468750,0.000
2,Cerebellum_White_Matter,0,2,0.178571,0.000000,0.781250,0.000
3,Cerebellum_White_Matter,0,3,0.107143,0.000000,0.812500,0.000
4,Cerebellum_White_Matter,1,0,0.925000,0.666667,0.954545,1.000


In [3]:
df_test_best_metrics = result = df_test_metrics.loc[df_test_metrics.groupby(['Brainpart', 'Pathogen'])['AUROC'].idxmax()]

# Display Dataframe
df_test_best_metrics.head()

,Brainpart,Pathogen,Test_Fold,AUROC,F1_Score,Acc,Prc
145,Accumbens_area,0,1,0.866071,0.000000,0.875000,0.0
148,Accumbens_area,1,0,0.950000,0.500000,0.909091,0.5
155,Accumbens_area,2,3,0.791667,0.285714,0.722222,1.0
158,Accumbens_area,3,2,0.755848,0.500000,0.851064,0.7
129,Amygdala,0,1,0.758929,0.444444,0.843750,0.4


In [4]:
df_test_best_metrics.to_csv('test_best_metrics.csv', index=False)